# Logic in Sole.jl

In [ ]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
Pkg.update()

## SoleLogics.jl

`SoleLogics.jl` is not only the package in the `Sole.jl` framework specifically
developed for logic: it is the core library of the framework itself.

In a nutshell, it provides a fresh codebase for computational logic, featuring
easy manipulation of:
- Propositional and (multi)modal logics (atoms, logical constants, alphabets,
grammars, crisp/fuzzy algebras);
- Logical formulas (parsing, random generation, minimization);
- Logical interpretations (propositional valuations, Kripke structures);
- Algorithms for finite
[model checking](https://en.wikipedia.org/wiki/Model_checking), that is,
checking that a formula is satisfied by an interpretation.

In this notebook, we will see examples of all these functionalities, providing
a comprehensive overview of the whole package.

In [ ]:
using SoleLogics

### Propositional Logic

Parsing and manipulating Formulas.

In [ ]:
φ1 = parseformula("¬p∧q∨(s∨z)")

In [ ]:
syntaxstring(φ1; parenthesize_commutatives = true)

In [ ]:
filter(ψ -> height(ψ) == 1, subformulas(φ1))

In [ ]:
filter(ψ -> natoms(ψ) == 1, subformulas(φ1))

In [ ]:
φ2 = ⊥ ∨ Atom("t") → φ1

Generating random formulas.

In [ ]:
alphabet = @atoms p q

In [ ]:
SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES

In [ ]:
using Random

h = 2;   # the height of the formula

randformula(
    Random.MersenneTwister(507),    # the random number generator we want to use
    h,
    alphabet,
    SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES
)

Model checking.

In [ ]:
φ1 = parseformula("¬(p ∧ q)")

In [ ]:
I = TruthDict(["p" => true, "q" => false])

In [ ]:
check(φ1, I)

In [ ]:
φ2 = parseformula("¬(p ∧ q) ∧ (r ∨ q)")

In [ ]:
interpret(φ2, I)

### Modal Logic K

Generating random formulas.

In [ ]:
SoleLogics.BASE_MODAL_CONNECTIVES

In [ ]:
randformula(
    Random.MersenneTwister(4267),
    h,
    alphabet,
    SoleLogics.BASE_MODAL_CONNECTIVES
)

Model checking.

In [ ]:
using Graphs

# Instantiate a Kripke frame with 5 worlds and 5 edges
worlds = World.(1:5)
edges = Edge.([(1,2), (1,3), (2,4), (3,4), (3,5)])
fr = SimpleModalFrame(worlds, Graphs.SimpleDiGraph(edges))

In [ ]:
# Enumerate the worlds that are accessible from the first world
accessibles(fr, first(worlds))

In [ ]:
# Assign each world a propositional interpretation
@atoms p q
valuation = Dict([
    worlds[1] => TruthDict([p => true, q => false]),
    worlds[2] => TruthDict([p => true, q => true]),
    worlds[3] => TruthDict([p => true, q => false]),
    worlds[4] => TruthDict([p => false, q => false]),
    worlds[5] => TruthDict([p => false, q => true]),
])

# Instantiate a Kripke structure by combining a Kripke frame and the
# propositional interpretations over each world
K = KripkeStructure(fr, valuation)

In [ ]:
# Generate a modal formula
φ = parseformula("◊(p ∧ q)");

# Check the just generated formula on each world of the Kripke structure
[w => check(φ, K, w) for w in worlds]

### Temporal Modal Logics

Linear Temporal Logic (LTL).

In [ ]:
# A frame consisting of 10 (evenly spaced) points
fr = FullDimensionalFrame((10,), Point{1, Int64})
allworlds(fr) |> collect

In [ ]:
# Linear Temporal Logic (LTL) `successor` relation
accessibles(fr, Point(3), SoleLogics.SuccessorRel) |> collect

In [ ]:
# Linear Temporal Logic (LTL) `greater than` (i.e., future) relation
accessibles(fr, Point(3), SoleLogics.GreaterRel) |> collect

Halpern and Shoham's Interval Temporal Logic (HS).

In [ ]:
# An interval frame consisting of all intervals over 10 (evenly spaced) points
fr = FullDimensionalFrame((10, ), Interval{Int64})
allworlds(fr) |> collect

In [ ]:
# Interval Algebra (IA) relation `L` (later)
accessibles(fr, Interval(3, 5), IA_L) |> collect

### Fuzzy Logic

In standard fuzzy logics, instead of constraining ourselves to only `true` and
`false` values, we let them be anythig between the continuous interval [0, 1].

On the downside, we cannot use the classical evaluation for the propositional
operators - what does it mean to be `0.3 and 0.5`?

Fuzzy logics are defined over a `t-norm` operation, which will be our
conjunction ($\wedge$); we have 3 of them(*):
- Goedel Logic, where the $tnorm(x, y)$ is defined as the $min\{x, y\}$
- Lukasiewicz Logic, where the $tnorm(x, y)$ is defined as the
$max\{0, x+y-1\}$
- Product Logic, where the $tnorm(x, y)$ is defined as the arithmetic product
$x \cdot y$

For each logic, the implication $x \to y$ will be defined as the
$max\{z | tnorm(x, z) \leq y\}$.

This can feel overwhelming at first, as it is very general: the important part
is that we can derive the implication for each fuzzy logic from their `t-norm`.

Finally, we will consider the disjunction between two values $x$ and $y$ simply
as the $max\{x, y\}$ for all logics.

(*) All other t-norms (hence, fuzzy logics) can be derived though a linear
combination of these 3.

In [ ]:
using SoleLogics.ManyValuedLogics   # fuzzy logics are defined in this submodule

In [ ]:
GodelLogic

In [ ]:
bot(GodelLogic)

In [ ]:
top(GodelLogic)

In [ ]:
uknown = ContinuousTruth(0.5)

The $tnorm(x, y)$ for Goedel Logic is defined as the $min\{x, y\}$.

In [ ]:
GodelLogic.tnorm(
    uknown,
    uknown
)

In [ ]:
LukasiewiczLogic

The $tnorm\{x, y\}$ for Lukasiewicz Logic is defined as the $max\{0, x+y-1\}.

In [ ]:
LukasiewiczLogic.tnorm(
    uknown,
    uknown
)

The $tnorm\{x, y\}$ for Product Logic is defined as the arithmetic product
$x \cdot y$.

In [ ]:
ProductLogic.tnorm(
    uknown,
    uknown
)

### Many-Valued Logic

While in fuzzy logics we consider a total order between all the values,
many-valued logics go even a step further: we also take into condiseration
values which can also be non-comparable, i.e., partial orders.

To do so, we leverage algebraic structures comprising lattices, such as Heyting
Algebras.

In [ ]:
using SoleLogics.ManyValuedLogics: G4, Ł4, H4
using SoleLogics.ManyValuedLogics: α, β

In [ ]:
getdomain(G4)

In [ ]:
getdomain(Ł4)

In [ ]:
getdomain(H4)

In [ ]:
precedes(G4, α, β)

In [ ]:
precedes(Ł4, α, β)

In [ ]:
precedes(H4, α, β)

In [ ]:
precedes(H4, ⊥, α)

In [ ]:
precedes(H4, ⊥, β)

In [ ]:
precedes(H4, α, ⊤)

In [ ]:
precedes(H4, β, ⊤)

While fuzzy logics differ on the `t-norm`, many-valued logics in general are
defined, among other things (like the set of values), over a more general
structure, called a `monoid`, that we will use to interpret conjunction
($\wedge$).

This coincide with the `t-norm` for fuzzy logics.

In [ ]:
G4.monoid(α, β)

In [ ]:
Ł4.monoid(α, β)

Heyting algebras can be thought of as a generalization of Goedel algebras (resp.
Goedel Logic) to partial orders: instead of taking the $min\{x, y\}$, which is
not always possible, we take the $inf\{x, y\}$, .e., the greatest lower bound.

For instance, since in our case $\alpha$ and $\beta$ are non-comparable, but
both greater than $\bot$ (and bigger than $\top$), their $inf$ will be $\bot$.

An $inf$ and a $sup$ is always guaranteed to exist (and to be unique) by
definition of a lattice.

In [ ]:
H4.monoid(α, β)

Let's have a quick look at a more comples example.

The following is also an Heyting algebras, but with 9 values.

In [ ]:
using SoleLogics.ManyValuedLogics: H9
using SoleLogics.ManyValuedLogics: ζ, η

getdomain(H9)

Let's take, for example, values $\zeta$ and $\eta$: they are non-comparable,
but they are both bigger than $\bot, \alpha, \beta, \delta$.

In [ ]:
using SoleLogics.ManyValuedLogics: lesservalues

lesservalues(H9, ζ)

In [ ]:
lesservalues(H9, η)

In [ ]:
intersect(lesservalues(H9, ζ), lesservalues(H9, η))

Hence, the greatest of these values ($\delta$) will be our result.

In [ ]:
H9.monoid(ζ, η)

**Exercise**: try to do the same with other values from the H9 algebra, such as:
- $\zeta$ and $\epsilon$
- $\zeta$ and $\beta$

Disjunction ($\vee$) and implication ($\to$) are generalized in a similar way:
- to evaluate a disjunction $x \vee y$, we will use the $sup\{x, y\}$ (lowest
greater bound)
- to evaluate an implication $x \to y$, we will use the
$sup\{z | monoid(x, z) \preceq y\}$

## SoleReasoners.jl